<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [176]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn import metrics
!pip install talos
import talos

In [177]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [178]:
train_df.shape

(7613, 2)

In [179]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [180]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model

In [181]:
p = {
    'first neuron units': (2,20,1),
    'first neuron activation': ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'epochs': [1,2,3,4,5,6,7,10,20,100],
    'batch size': (5,100,5)
}

In [182]:
#loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

In [183]:
x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(7613,)

In [184]:
#experiment_data = t.data
#experiment_data.head()

In [185]:
#experiment_data['val_accuracy'].idxmax()

In [186]:
#experiment_data.iloc[205]

In [187]:
#vectorizer = CountVectorizer()
#vectorizer.fit(train_df['text'])

#data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [188]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

#x_train, x_test, y_train, y_test = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

train_sequences = tokenizer.texts_to_sequences(x_train)
validation_sequences = tokenizer.texts_to_sequences(x_validation)
train_data = pad_sequences(train_sequences, maxlen=100)
validation_data = pad_sequences(validation_sequences, maxlen=100)


#input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Embedding(5000, 128, input_length=100))
best_model.add(Conv1D(512, 5, activation='relu'))
best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(MaxPooling1D(5))
#best_model.add(layers.Dropout(0.25))
#best_model.add(Flatten())
#best_model.add(LSTM(64))
best_model.add(layers.Dense(128 ,activation='relu'))
#best_model.add(layers.Dense(128 ,activation='relu'))
best_model.add(layers.Dense(1,activation='sigmoid'))
#best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(train_data,y_train,epochs = 5,verbose = 1,validation_data = (validation_data,y_validation),batch_size=62)

loss, accuracy = best_model.evaluate(train_data, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model.evaluate(validation_data, y_validation, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 7613 samples, validate on 1523 samples
Epoch 1/5
7613/7613 [==============================] - 42s 6ms/step - loss: 0.5659 - accuracy: 0.6997 - val_loss: 0.3927 - val_accuracy: 0.8477
Epoch 2/5
7613/7613 [==============================] - 39s 5ms/step - loss: 0.3741 - accuracy: 0.8466 - val_loss: 0.2968 - val_accuracy: 0.8910
Epoch 3/5
7613/7613 [==============================] - 40s 5ms/step - loss: 0.2993 - accuracy: 0.8869 - val_loss: 0.2432 - val_accuracy: 0.9140
Epoch 4/5
7613/7613 [==============================] - 40s 5ms/step - loss: 0.2499 - accuracy: 0.9078 - val_loss: 0.1864 - val_accuracy: 0.9402
Epoch 5/5
7613/7613 [==============================] - 40s 5ms/step - loss: 0.2068 - accuracy: 0.9247 - val_loss: 0.1503 - val_accuracy: 0.9514
Training Accuracy: 0.9476
Testing Accuracy:  0.9514


In [189]:
prediction=best_model.predict_classes(validation_data, batch_size=512)
score = roc_auc_score(y_validation, prediction)
score

0.9472509658199733

In [190]:
accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

0.9514116874589625

In [191]:
#Reentrena con el set completo y realiza predicción.

#set_sequences = tokenizer.texts_to_sequences(train_df['text'])
#set_data = pad_sequences(set_sequences, maxlen=100)

#out = best_model.fit(set_data,train_df['target'],epochs = 5,verbose = 1,batch_size=62)

#out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
#data_test = vectorizer.transform(test_df['text']).toarray()
test_prediction = tokenizer.texts_to_sequences(test_df['text'])
prediction_data = pad_sequences(test_prediction, maxlen=100)

prediction2=best_model.predict_classes(prediction_data, batch_size=512)
prediction2

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [192]:
# Genera archivo submit.

test_df['target'] = prediction2
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>